# Getting some quick picks for 2023-04-08

http://ufcstats.com/event-details/3dc3022232b79c7a

https://www.bestfightodds.com/events/ufc-287-2760

Unfortunately I forgot to run the full script prior to this, so it's a bit jank

In [ ]:
import sys
sys.path.append('~/Documents/code/sports') # add parent directory to path

: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from db import base_db_interface

df = base_db_interface.read("bfo_espn_ufc_features")
for dt_col in ["Date", "DOB", "DOB_opp"]:
    df[dt_col] = pd.to_datetime(df[dt_col])
df[["FighterOpen", "OpponentOpen"]] = df[["FighterOpen", "OpponentOpen"]]\
    .astype(float)

df = df.drop_duplicates(subset=["FighterID_espn", "OpponentID_espn", "fight_id"])
df["FighterID_espn"] = df["FighterID_espn"].fillna("unknown")
df["OpponentID_espn"] = df["OpponentID_espn"].fillna("unknown")
print(df.shape)
df.head() # show the first 5 rows

: 

In [ ]:
from sklearn.decomposition import PCA
# OKAY PCA HAPPENS HERE. Just do everything in-sample for now. 
stat_cols = [
    'TSL', 'TSA', 'SSL',
    'SSA', #'TSL-TSA', 
    'KD', #'%BODY', '%HEAD', '%LEG', 
    'SCBL',
    'SCBA', 'SCHL', 'SCHA', 'SCLL', 'SCLA', 'RV', 'TDL', 'TDA', 'TDS',
    # 'TK ACC', 'SR', # I don't believe in ratio features in PCA, 
    # # because of the possibility of division by zero and heteroskedasticity
    'SGBL', 'SGBA', 'SGHL', 'SGHA', 'SGLL', 'SGLA', 'AD', 'ADTB',
    'ADHG', 'ADTM', 'ADTS', 'SM', 'SDBL', 'SDBA', 'SDHL',
    'SDHA', 'SDLL', 'SDLA',
    #'time_seconds',
    # 'TD_fails', #'submission_rate',
    'TD_fail', # formerly 'TD_fails'
    'SDL', 'SCL', # formerly 'distance_strikes_landed', 'clinch_strikes_landed',
    #'KD_power', 
    'SGL', # formerly 'ground_strikes_landed'
]
df[["KD", "KD_opp"]] = df[["KD", "KD_opp"]].astype(float) 
# convert from string to float. rather annoying

: 

In [ ]:
stat_diff_df = {
    "FighterID_espn": df["FighterID_espn"],
    "OpponentID_espn": df["OpponentID_espn"],
    "Date": df["Date"],
    "gender": df["gender"],
    "fight_id": df["fight_id"],
}
diff_cols = [col+"_diff" for col in stat_cols]
for col, diff_col in zip(stat_cols, diff_cols):
    stat_diff_df[diff_col] = (
        np.sqrt(df[col]) - np.sqrt(df[col+"_opp"])
    )
# stat_diff_df = pd.DataFrame(stat_diff_df).dropna(subset=diff_cols).reset_index()
stat_diff_df = pd.DataFrame(stat_diff_df).reset_index()

: 

# Elo estimators

In [ ]:
n_pca = 14
# n_pca = 1 # just for testing
bin_elo_alpha = 0.45
acc_elo_alpha = 0.45
pca_elo_alpha = 0.45
real_elo_alpha = 0.45

: 

In [ ]:
from importlib import reload
## use autoreload jupyter extension

import model.mma_elo_model as mma_elo_model
reload(mma_elo_model)

import model.mma_features as mma_features
reload(mma_features)

: 

In [ ]:
from model.mma_features import PcaEloWrapper, BinaryEloWrapper

pca_ew = PcaEloWrapper(
    n_pca=n_pca, target_cols=diff_cols, alpha=pca_elo_alpha,
    conditional_var_col=None, # for consistency
)
pca_elo_feat_df = pca_ew.fit_transform_all(stat_diff_df)
pca_elo_feat_df

: 

In [ ]:
pca_elo_feat_df.notnull().sum(), stat_diff_df.notnull().sum()

: 

## Defining some targets for Elo estimators along the way

In [ ]:
df["win_target"] = df["FighterResult"].replace({"W":1, "L":0, "D":np.nan})
df["win_target"].value_counts(dropna=False)

: 

In [ ]:
# have to parse the decision column to get the detailed result
def parse_decision(s):
    if (s.startswith("submission") or 
        s.startswith("sumission") or
        s.startswith("technical submission")
    ):
        return "submission"
    if (s.startswith("tko") or 
        s.startswith("ko") or
        (s == 'could not continue')
    ):
        return "tko_ko"
    if "decision" in s:
        return "decision"
    return "other"

temp_decision = df["Decision"].fillna("-").str.lower().str.strip()
decision_clean = temp_decision.apply(parse_decision)
result_sign = df["FighterResult"].map({"W": 1, "L":-1, "D": 0})
decision_score = decision_clean.map({"tko_ko":2, "submission":2, 
                                                    "decision":1, "other":0})
df["ordinal_fighter_result"] = result_sign * decision_score
submission_score = decision_clean.map({"submission":1, "decision":0, 
                                            "other":0, "tko_ko":0})
tko_ko_score = decision_clean.map({"submission":0, "decision":0, 
                                            "other":0, "tko_ko":1})
decision_score = decision_clean.map({"submission":0, "decision":1, 
                                            "other":0, "tko_ko":0})
finish_score = decision_clean.map({"submission":1, "decision":0, 
                                            "other":0, "tko_ko":1})
df["submission_fighter_result"] = result_sign * submission_score
df["tko_ko_fighter_result"] = result_sign * tko_ko_score
df["decision_fighter_result"] = result_sign * decision_score
df["finish_fighter_result"] = result_sign * finish_score

df["win_target_finish"] = df["win_target"] * decision_clean.map({
    "submission":1, "tko_ko":1,
    "decision":np.nan, "other":np.nan, 
})
df["win_target_finish"].value_counts(dropna=False)

: 

In [ ]:
def parse_fight_time(row):
    if not np.isnan(row["time_dur"]):
        return row["time_dur"]
    if row["Rnd"] == "-":
        return np.nan
    if row["Time"] == "-":
        # assuming 5 minute rounds
        # fill in something for now
        return int(row["Rnd"]) * 5 * 60
    n_rounds = row["Rnd"].strip()
    min, sec = row["Time"].strip().split(":")
    if n_rounds == "-":
        return np.nan
    n_rounds = int(n_rounds) - 1
    if (min == "-") | (sec == "-"):
        return np.nan
    min = int(min)
    sec = int(sec)
    # assuming 5 minute rounds
    return (n_rounds * 5 * 60) + min * 60 + sec
    
df["Rnd"] = df["Rnd"].fillna("-")
df["Time"] = df["Time"].fillna("-")
df["fight_time"] = df.apply(parse_fight_time, axis=1)
df["fight_time"].notnull().mean()

: 

In [ ]:
y = (-1)**df["win_target"]
# restrict btw 1 and 25 minutes (length of a title fight)
# short fight times -> big outliers
clipped_time_dur = df["fight_time"].clip(60, (5*5*60))
df["signed_inverse_fight_time"] = y / clipped_time_dur
df["signed_inverse_sqrt_fight_time"] = y / np.sqrt(clipped_time_dur)
df["signed_inverse_log_fight_time"] = y / np.log(clipped_time_dur)

df["signed_inverse_fight_time"].hist(bins=100)

: 

In [ ]:
from model.mma_features import RealEloWrapper

real_elo_target_cols = [
#     "fighter_result_time_left", 
#     "ml_logit_mvmt",
    # "ordinal_fighter_result",
    # "submission_fighter_result",
    # "tko_ko_fighter_result",
    # "decision_fighter_result",
    # "finish_fighter_result",
    "signed_inverse_fight_time",
    # "signed_inverse_sqrt_fight_time",
    # "signed_inverse_log_fight_time",
]
diff_elo_target_cols = [
]

binary_elo_target_cols = ["win_target", "win_target_finish"]

elo_alphas = {
    col: real_elo_alpha 
    for col in (real_elo_target_cols + diff_elo_target_cols)
}
# elo_alphas["ml_logit_mvmt"] = 0.225

real_ew = RealEloWrapper(elo_alphas=elo_alphas)
real_elo_feat_df = real_ew.fit_transform_all(df)
print(real_elo_feat_df.shape)
real_elo_feat_df.head()

elo_alphas = {
    col: bin_elo_alpha for col in binary_elo_target_cols
}
bin_ew = BinaryEloWrapper(elo_alphas=elo_alphas)
bin_elo_feat_df = bin_ew.fit_transform_all(df)
print(bin_elo_feat_df.shape)
bin_elo_feat_df.head()
# feat_ml_df["log_height_diff"] = np.log(feat_ml_df["imp_height"]) - np.log(feat_ml_df["imp_height_opp"])
# feat_ml_df["log_age_diff"] = np.log(feat_ml_df["age"]) - np.log(feat_ml_df["age_opp"])
# feat_ml_df["log_reach_diff"] = np.log(feat_ml_df["imp_reach"]) - np.log(feat_ml_df["imp_reach_opp"])
# feat_ml_df["log_reach_diff"] = feat_ml_df["log_reach_diff"].fillna(0)

: 

In [ ]:
elo_feat_df = df.merge(
    pca_elo_feat_df, 
    how="left", 
    on=["fight_id", "FighterID_espn", "OpponentID_espn"],
).merge(
    real_elo_feat_df.drop(columns=real_elo_target_cols),
    how="left",
    on=["fight_id", "FighterID_espn", "OpponentID_espn"],
).merge(
    bin_elo_feat_df.drop(columns=binary_elo_target_cols),
    how="left",
    on=["fight_id", "FighterID_espn", "OpponentID_espn"],
)
print(elo_feat_df.shape)

: 

In [ ]:
elo_feat_df.notnull().sum()

: 

# Simpler features

In [ ]:
from model.mma_elo_model import unknown_fighter_id
from tqdm import tqdm


def get_simple_features(df):
    # simple things that i needn't get from the fighter stats page
    # eg number of fights, t_since_last_fight
    assert (df["fight_id"].value_counts() == 2).all()
    df = df.assign(
        is_ufc=df["Event"].fillna("").str.contains("UFC"),
        Date=pd.to_datetime(df["Date"]),
    )
    feat_df = df.sort_values("Date").copy()[[
        "fight_id", "FighterID_espn", "OpponentID_espn", "Date", "is_ufc"
    ]]
    # Rolling features over fighter_careers
    # because the data is doubled, we can simply group by the fighter id
    feat_df["dummy"] = 1
    # total fights
    feat_df["total_fights"] = feat_df.groupby("FighterID_espn")["dummy"].cumsum()
    feat_df["total_fights_opp"] = feat_df.groupby("OpponentID_espn")["dummy"].cumsum()
    # total ufc fights
    feat_df["total_ufc_fights"] = feat_df.groupby("FighterID_espn")["is_ufc"].cumsum()
    feat_df["total_ufc_fights_opp"] = feat_df.groupby("OpponentID_espn")["is_ufc"].cumsum()
    # time since last fight
    feat_df["t_since_last_fight"] = feat_df.groupby("FighterID_espn")["Date"].diff().dt.days
    feat_df["t_since_last_fight_opp"] = feat_df.groupby("OpponentID_espn")["Date"].diff().dt.days
    fill_val = 2*365 # arbitrarily say 2 years
    feat_df["t_since_last_fight"] = np.maximum(fill_val, feat_df["t_since_last_fight"].fillna(fill_val))   
    feat_df["t_since_last_fight_opp"] = np.maximum(fill_val, feat_df["t_since_last_fight_opp"].fillna(fill_val)) 
    # time since first fight
    feat_df["t_since_first_fight"] = (feat_df["Date"] - feat_df.groupby("FighterID_espn")["Date"].transform("min")).dt.days
    feat_df["t_since_first_fight_opp"] = (feat_df["Date"] - feat_df.groupby("OpponentID_espn")["Date"].transform("min")).dt.days
    # compute diffs
    feat_df["t_since_last_fight_diff"] = (feat_df["t_since_last_fight"] - 
                                            feat_df["t_since_last_fight_opp"])
    feat_df["t_since_last_fight_log_diff"] = (np.log(feat_df["t_since_last_fight"]) - 
                                                np.log(feat_df["t_since_last_fight_opp"]))
    feat_df["total_fights_diff"] = (feat_df["total_fights"] - 
                                    feat_df["total_fights_opp"])
    feat_df["total_fights_sqrt_diff"] = (np.sqrt(feat_df["total_fights"]) - 
                                        np.sqrt(feat_df["total_fights_opp"]))
    feat_df["total_ufc_fights_diff"] = (feat_df["total_ufc_fights"] - 
                                        feat_df["total_ufc_fights_opp"])
    feat_df["total_ufc_fights_sqrt_diff"] = (np.sqrt(feat_df["total_ufc_fights"]) - 
                                                np.sqrt(feat_df["total_ufc_fights_opp"]))
    return feat_df

simple_feat_df = get_simple_features(elo_feat_df)
print(simple_feat_df.shape, elo_feat_df.shape)

: 

In [ ]:
simple_feat_df.columns

: 

In [ ]:
feat_ml_df = elo_feat_df.merge(
    simple_feat_df, 
    how="left",
    on=["FighterID_espn", "OpponentID_espn", "Date"],
    suffixes=("_legacy", ""),
)

feat_ml_df["age_diff"] = (feat_ml_df["DOB"] - feat_ml_df["DOB_opp"]).dt.days / 365
feat_ml_df["age_diff"] = feat_ml_df["age_diff"].fillna(0)

feat_ml_df["log_reach_diff"] = (
    np.log(feat_ml_df["ReachInches"]) - np.log(feat_ml_df["ReachInches_opp"])
).fillna(0)
feat_ml_df["weight_diff"] = (
    feat_ml_df["min_weight"] - feat_ml_df["min_weight_opp"]
).fillna(0)
feat_ml_df["height_diff"] = (
    feat_ml_df["HeightInches"] - feat_ml_df["HeightInches_opp"]
).fillna(0)

feat_ml_df["t_since_first_fight_diff"] = (
    feat_ml_df["t_since_first_fight"] - feat_ml_df["t_since_first_fight_opp"]
).fillna(0)
feat_ml_df["log_t_since_first_fight_diff"] = (
    np.log(1 + feat_ml_df["t_since_first_fight"]) - 
    np.log(1 + feat_ml_df["t_since_first_fight_opp"])
).fillna(0)

feat_ml_df["log_t_since_prev_fight_diff"] = (
    np.log(1 + feat_ml_df["t_since_prev_fight"]) -
    np.log(1 + feat_ml_df["t_since_prev_fight_opp"])
).fillna(0)

real_elo_target_cols = [
#     "fighter_result_time_left", 
#     "ml_logit_mvmt",
#     "ordinal_fighter_result",
#     "submission_fighter_result",
#     "tko_ko_fighter_result",
#     "decision_fighter_result",
    "signed_inverse_fight_time",
    # "finish_fighter_result",
    # *[col for col in stat_pca_df.columns 
    #   if col not in ["FighterID_espn", "OpponentID_espn", "Date"]],
]

feat_cols = [
    *[f"pred_elo_PC_{i}" for i in range(n_pca)],

    *["pred_elo_"+c for c in [*diff_elo_target_cols, 
                               *real_elo_target_cols, 
                               *binary_elo_target_cols]],
    
    # "t_since_last_fight_log_diff", 
#     "fights_per_day_diff",
#     "t_since_last_fight_diff",
#     "total_fights_sqrt_diff", 
#     "total_ufc_fights_diff",
    
    "age_diff", 
#     "log_age_diff",
#     "reach_diff", 
    "log_reach_diff",
    "weight_diff", 
#     "log_weight_diff",
    "height_diff",
#     "log_height_diff",
#     "ml_logit_mvmt",
    "log_t_since_prev_fight_diff",
    "log_t_since_first_fight_diff",
    "total_fights_diff",
#     "quad_log_t_since_first_fight_diff",
]

# new_feat_cols = [*feat_cols, "ml_logit"]
feat_cols

: 

In [ ]:
feat_ml_df.columns[feat_ml_df.columns.str.contains("pred_")]

: 

In [ ]:
# checking to see how much data we have for each feature
feat_ml_df[feat_cols].notnull().sum()

: 

In [ ]:
feat_ml_df = feat_ml_df.drop_duplicates(subset=["fight_id"])
print(feat_ml_df[feat_cols].notnull().sum())
feat_ml_df.shape

: 

# Model evaluation - opening line

In [ ]:
p_fighter_implied_col = "p_fighter_open_implied"

: 

In [ ]:
from model.mma_log_reg_stan import SimpleSymmetricModel

mod = SimpleSymmetricModel(
    feat_cols=feat_cols, target_col="win_target", 
    p_fighter_implied_col=p_fighter_implied_col,
    beta_prior_std=1.0, mcmc=False
)

: 

In [ ]:
from model_selection.cross_val_pipeline import TimeSeriesCrossVal

tscv = TimeSeriesCrossVal(min_test_date=pd.to_datetime("2021-01-01"), 
                          n_dates_per_fold=1, 
                          p_fighter_implied_col=p_fighter_implied_col)
preds_df = tscv.get_cross_val_preds(
    mod, 
    feat_ml_df.dropna(subset=[
        *feat_cols, "win_target", p_fighter_implied_col,
    ])
)

: 

In [ ]:
mod_pred = preds_df["y_pred"].round()
ml_pred = preds_df[p_fighter_implied_col].round()
print("Model accuracy:     ", (mod_pred == preds_df["win_target"]).mean())
print("Moneyline accuracy: ", (ml_pred == preds_df["win_target"]).mean())

: 

In [ ]:
from sklearn.metrics import log_loss

xce = log_loss(y_true=preds_df["win_target"], y_pred=preds_df["y_pred"])
xce_ml = log_loss(y_true=preds_df["win_target"], y_pred=preds_df[p_fighter_implied_col])

print(f"model log loss    : {xce}")
print(f"Moneyline log loss: {xce_ml}")

: 

In [ ]:
list(zip(mod.feat_cols, mod.fit['beta']))

: 

In [ ]:
preds_df["week"] = preds_df["Date"].dt.isocalendar().week

: 

In [ ]:
fighter_ml_col = "FighterOpen"
opponent_ml_col = "OpponentOpen"

: 

In [ ]:
from model_selection.metrics import MultiKellyPM

kwargs = {
    "max_bankroll_fraction": 0.05,
    "groupby_col": "week",
    "fighter_ml_col": fighter_ml_col,
    "opponent_ml_col": opponent_ml_col,
}

print("overall winnings")
pm = MultiKellyPM(preds_df, **kwargs)
event_return_df = pm.get_all_returns()
pm.plot_diagnostics(event_return_df, x_col="week")

print("ufc winnings")
pm = MultiKellyPM(preds_df.query("is_ufc == 1"), **kwargs)
event_return_df = pm.get_all_returns()
pm.plot_diagnostics(event_return_df, x_col="week")

print("non-ufc winnings")
pm = MultiKellyPM(preds_df.query("is_ufc == 0"), **kwargs)
event_return_df = pm.get_all_returns()
pm.plot_diagnostics(event_return_df, x_col="week")

: 

# Model evaluation - BetMGM, DraftKings, BetFair

I think something's fishy here, this can't be right

In [ ]:
close_df = base_db_interface.read("clean_bfo_close_data")
# double the data
fighter_cols = [c for c in close_df.columns if c.endswith("_fighter")]
opponent_cols = [c[:-len("_fighter")] + "_opponent" for c in fighter_cols]
for col in fighter_cols + opponent_cols:
    close_df[col] = close_df[col].str.replace("▲", "")\
        .str.replace("▼", "")\
        .astype(float)
    
close_df_complement = close_df.rename(columns={
    "FighterID": "OpponentID",
    "OpponentID": "FighterID",
    **{f: o for f, o in zip(fighter_cols, opponent_cols)},
    **{o: f for f, o in zip(fighter_cols, opponent_cols)},
})
close_df = pd.concat([close_df, close_df_complement], axis=0)\
    .drop_duplicates(subset=["FighterID", "OpponentID", "EventHref"])\
    .dropna(subset=["FighterID", "OpponentID", "EventHref"])\
    .rename(columns={"FighterID": "FighterID_bfo", "OpponentID": "OpponentID_bfo"})\
    .drop(columns=["FighterName", "OpponentName"])\
    .reset_index(drop=True)

print(close_df.shape)
close_df.isnull().mean()

: 

In [ ]:
preds_df["FighterID_bfo"]

: 

In [ ]:
aug_preds_df = preds_df.merge(
    close_df,
    how="left",
    on=["FighterID_bfo", "OpponentID_bfo", "EventHref"],
)
aug_preds_df.isnull().mean()

: 

In [ ]:
close_df[["DraftKings_fighter", "BetMGM_fighter"]]

: 

In [ ]:
preds_df[fighter_ml_col]

: 

In [ ]:
market = "DraftKings"
# market = "BetMGM"

kwargs = {
    "max_bankroll_fraction": 1,
    "groupby_col": "week",
    "fighter_ml_col": f"{market}_fighter",
    "opponent_ml_col": f"{market}_opponent",
}

temp_preds_df = aug_preds_df.dropna(subset=[
    f"{market}_fighter", f"{market}_opponent"
])
temp_preds_df

print("overall winnings")
pm = MultiKellyPM(temp_preds_df, **kwargs)
event_return_df = pm.get_all_returns()
pm.plot_diagnostics(event_return_df, x_col="week")

print("ufc winnings")
pm = MultiKellyPM(temp_preds_df.query("is_ufc == 1"), **kwargs)
event_return_df = pm.get_all_returns()
pm.plot_diagnostics(event_return_df, x_col="week")

print("non-ufc winnings")
pm = MultiKellyPM(temp_preds_df.query("is_ufc == 0"), **kwargs)
event_return_df = pm.get_all_returns()
pm.plot_diagnostics(event_return_df, x_col="week")

: 

# Predictions for upcoming fights

In [ ]:
mod = SimpleSymmetricModel(
    feat_cols=feat_cols, target_col="win_target", 
    p_fighter_implied_col=p_fighter_implied_col,
    beta_prior_std=1.0, mcmc=False
)
preds_df = tscv.get_cross_val_preds(
    mod, 
    feat_ml_df.dropna(subset=[
        *feat_cols, "win_target", p_fighter_implied_col,
    ])
)

: 

In [ ]:
from wrangle.clean_bfo_data import parse_american_odds

main_card_df = [
    ("Israel Adesanya", "Alex Pereira", "-150", "+130"),
    ("Gilbert Burns", "Jorge Masvidal", "-500", "+400"),
    ("Adrian Yanez", "Rob Font", "-185", "+160"),
    ("Kevin Holland", "Santiago Ponzinibbio", "-200", "+170"),
    ("Christian Rodriguez", "Raul Rosas Jr.", "+210", "-250"),
]
main_card_df = pd.DataFrame(
    main_card_df, 
    columns=["FighterName_ufc", "OpponentName_ufc", 
             "FighterOpen", "OpponentOpen"],
)
main_card_df_complement = main_card_df.rename(columns={
    "FighterName_ufc": "OpponentName_ufc",
    "OpponentName_ufc": "FighterName_ufc",
    "FighterOpen": "OpponentOpen",
    "OpponentOpen": "FighterOpen",
})
main_card_df = pd.concat([
    main_card_df,
    main_card_df_complement,
]).reset_index(drop=True)
main_card_df

: 

In [ ]:
train_df = feat_ml_df.query("is_ufc == 1 & is_upcoming == 0")\
    .dropna(subset=[
        *feat_cols, "win_target", p_fighter_implied_col
    ])
test_df = feat_ml_df.query("is_upcoming == 1 & Date == '2023-04-08'").copy()
test_df = test_df.drop(columns=["FighterOpen", "OpponentOpen"]).merge(
    main_card_df,
    how="inner",
    on=["FighterName_ufc", "OpponentName_ufc"],
)
p_fighter = parse_american_odds(test_df["FighterOpen"])
p_opponent = parse_american_odds(test_df["OpponentOpen"])
test_df["p_fighter_open_implied"] = p_fighter / (p_fighter + p_opponent)
test_df
# test_df[p_fighter_implied_col] = test_df[p_fighter_implied_col].fillna(0.5)
train_df.shape, test_df.shape

: 

In [ ]:
test_df[["FighterName_ufc", "OpponentName_ufc"]]

: 

In [ ]:
bfo_to_ufc_map = base_db_interface.read("bfo_to_ufc_map")
bfo_to_ufc_map

: 

In [ ]:
ufc_df = close_df.query("EventHref == '/events/ufc-287-2760'")
print(ufc_df.shape)
ufc_df = ufc_df.merge(
    bfo_to_ufc_map[["FighterID_bfo", "FighterID_ufc"]].drop_duplicates(), 
    how="left", 
    on=["FighterID_bfo"]
).merge(
    bfo_to_ufc_map[["OpponentID_bfo", "OpponentID_ufc"]].drop_duplicates(),
    how="left",
    on=["OpponentID_bfo"]
)
print(ufc_df.shape)
ufc_df

: 

In [ ]:
aug_test_df = test_df.merge(
    ufc_df,
    how="left",
    on=["FighterID_ufc", "OpponentID_ufc"],
)
aug_test_df

: 

In [ ]:
# aug_test_df["p_fighter_open_implied"]# = aug_test_df["p_fighter_open_implied"].fillna(0.5)
aug_test_df.columns[aug_test_df.columns.str.contains("p_fighter_open_implied")]

: 

In [ ]:
mod = SimpleSymmetricModel(
    feat_cols=feat_cols, target_col="win_target", 
    p_fighter_implied_col=p_fighter_implied_col,
    beta_prior_std=1.0, mcmc=False
)

y_pred = mod.fit_predict(train_df, aug_test_df, feat_cols)
y_pred

: 

In [ ]:
from scipy.special import logit, expit
preds_df = aug_test_df.assign(
    y_pred = y_pred,
    # bet_dir = logit(y_pred) - logit(0.5)
)
preds_df[[
    "FighterName_ufc", "OpponentName_ufc", 
    p_fighter_implied_col, 
    "y_pred", "Date",
    # "DraftKings_fighter", "DraftKings_opponent",
    "BetMGM_fighter", "BetMGM_opponent",
    "Caesars_fighter", "Caesars_opponent",
    "FanDuel_fighter", "FanDuel_opponent",
]]

for col in [
        "BetMGM_fighter", "BetMGM_opponent",
    "Caesars_fighter", "Caesars_opponent",
    "FanDuel_fighter", "FanDuel_opponent",
    # "FighterOpen", "Opponent"
    ]:
    preds_df[col] = preds_df[col].astype(float)
preds_df

: 

In [ ]:
preds_df[["FighterName_ufc", "OpponentName_ufc", "y_pred", p_fighter_implied_col, "Date"]]

: 

In [ ]:
from model_selection.metrics import MultiKellyPM

pm = MultiKellyPM(preds_df,
                  fighter_ml_col="FanDuel_fighter",
                    opponent_ml_col="FanDuel_opponent",
                  parse_ml=True)
pm.get_portfolio_weights()

: 

: 